In [336]:
import pandas as pd
import numpy as np

In [337]:
import warnings
warnings.filterwarnings('ignore')

In [338]:
data = pd.read_csv('nano-particle-road exact.csv')

In [339]:
data

,Material,Output (per sq. meter),Cost (per sq. meter),Durability,Efficiency,Sensitivity,Maintenance,Scalability
0,Piezoelectric,0.05 W,$50,7.5 years,Medium,Low,Very Low,High
1,Triboelectric,0.01 W,$5,5.5 years,High,High,Low,Very High
2,Thermoelectric,0.55 W,$275,11.5 years,Medium,Medium,Low,Very High
3,Photovoltaic,105 W,$1.10,20 years,Medium,Very Low,High,Medium
4,Electroactive Polymers,0.05 W,$27.50,3.5 years,Medium,Very Low,Low,High
5,Magnetoelectric,0.05 W,$550,8.5 years,High,High,Low,Very High
6,Pyroelectric,0.05 W,$110,7.5 years,High,Medium,Very Low,High


In [340]:
df = data.copy()

In [341]:
values_tomap = {
    'Very High':5,
    'High' :4,
    'Medium':3,
    'Low':2,
    'Very Low':1
}

In [342]:
# Use only if the columns name are irreggular
def regularization():
    for col in data.columns:
        data.rename(columns = {col : col.strip()},inplace=True)

In [343]:
regularization()

In [344]:
# mapping five scale values
def mapping_five_scales(columns):
    for col in columns:
        data[col] = data[col].apply(lambda x:x.strip()).map(values_tomap)
    return data

In [345]:
mapping_five_scales(['Efficiency','Sensitivity','Maintenance','Scalability'])

,Material,Output (per sq. meter),Cost (per sq. meter),Durability,Efficiency,Sensitivity,Maintenance,Scalability
0,Piezoelectric,0.05 W,$50,7.5 years,3,2,1,4
1,Triboelectric,0.01 W,$5,5.5 years,4,4,2,5
2,Thermoelectric,0.55 W,$275,11.5 years,3,3,2,5
3,Photovoltaic,105 W,$1.10,20 years,3,1,4,3
4,Electroactive Polymers,0.05 W,$27.50,3.5 years,3,1,2,4
5,Magnetoelectric,0.05 W,$550,8.5 years,4,4,2,5
6,Pyroelectric,0.05 W,$110,7.5 years,4,3,1,4


In [346]:
# mapping the integer values
data['Cost (per sq. meter)']   = round(data['Cost (per sq. meter)'].str.replace('$','').astype('float64'),2)
data['Output (per sq. meter)'] = round(data['Output (per sq. meter)'].str.replace('W','').astype('float64'),2)
data['Durability']             = round(data['Durability'].str.replace('years','').astype('float64'),2)

In [347]:
data

,Material,Output (per sq. meter),Cost (per sq. meter),Durability,Efficiency,Sensitivity,Maintenance,Scalability
0,Piezoelectric,0.05,50.0,7.5,3,2,1,4
1,Triboelectric,0.01,5.0,5.5,4,4,2,5
2,Thermoelectric,0.55,275.0,11.5,3,3,2,5
3,Photovoltaic,105.00,1.1,20.0,3,1,4,3
4,Electroactive Polymers,0.05,27.5,3.5,3,1,2,4
5,Magnetoelectric,0.05,550.0,8.5,4,4,2,5
6,Pyroelectric,0.05,110.0,7.5,4,3,1,4


In [348]:
data.drop(columns=['Material'],inplace=True)

In [349]:
Beneficial = ['Output (per sq. meter)','Durability','Efficiency','Scalability']
Non_Beneficial = ['Cost (per sq. meter)','Sensitivity','Maintenance']

# Step 1 : Normalization matrix based on the Beneficial and Non_beneficial criteria

In [350]:
def min_max(df):
    df_n = df.copy()
    l = dict()
    for col in df_n.columns:
        l[col] = (min(df_n[col]),max(df_n[col]))
    return l

In [351]:
def beneficial_normalization(columns,df,min_max_d):
    df_n = df.copy()
    for col in columns:
        df_n[col] = (df_n[col] - min_max_d[col][0])/(min_max_d[col][1]-min_max_d[col][0])
    return df_n

In [352]:
def nonBeneficial_normalization(columns,df,min_max_d):
    df_n = df.copy()
    for col in columns:
        df_n[col] = (min_max_d[col][1]- df_n[col])/(min_max_d[col][1]-min_max_d[col][0])
    return df_n

In [353]:
min_max_data = min_max(data)
min_max_data

{'Output (per sq. meter)': (0.01, 105.0),
 'Cost (per sq. meter)': (1.1, 550.0),
 'Durability': (3.5, 20.0),
 'Efficiency': (3, 4),
 'Sensitivity': (1, 4),
 'Maintenance': (1, 4),
 'Scalability': (3, 5)}

In [354]:
norm_bene = beneficial_normalization(Beneficial,data,min_max_data)
norm_nonbene = nonBeneficial_normalization(Non_Beneficial,norm_bene,min_max_data)

In [355]:
norm_matrix = norm_nonbene.copy()
norm_matrix

,Output (per sq. meter),Cost (per sq. meter),Durability,Efficiency,Sensitivity,Maintenance,Scalability
0,0.000381,0.910913,0.242424,0.0,0.666667,1.000000,0.5
1,0.000000,0.992895,0.121212,1.0,0.000000,0.666667,1.0
2,0.005143,0.501002,0.484848,0.0,0.333333,0.666667,1.0
3,1.000000,1.000000,1.000000,0.0,1.000000,0.000000,0.0
4,0.000381,0.951904,0.000000,0.0,1.000000,0.666667,0.5
5,0.000381,0.000000,0.303030,1.0,0.000000,0.666667,1.0
6,0.000381,0.801603,0.242424,1.0,0.333333,1.000000,0.5


# Step 2 : Difference of Each Alternatives

In [356]:
# difference of each alternatives
def difference_mat(df):
    l=[]
    for i in range(0,df.shape[0]):
        for j in range(0,df.shape[0]):
            if i!=j:
                l.append(df.loc[i,]-df.loc[j,])
    df_new = pd.DataFrame(naming())
    df_new = pd.concat([df_new,pd.DataFrame(l)],axis=1)
    #df_new  = pd.DataFrame(l)
    return df_new

In [357]:
#creating material name columns
def naming():
    l=[]
    for i in range(norm_matrix.shape[0]):
        for j in range(norm_matrix.shape[0]):
            if i!=j:
                l.append(f'Nano - Material{(i,j)}')
    return l

In [358]:
dif_mat = pd.DataFrame()
dif_mat = difference_mat(norm_matrix)
dif_mat.rename(columns = {0:'Nano-materials'},inplace=True)
dif_mat

,Nano-materials,Output (per sq. meter),Cost (per sq. meter),Durability,Efficiency,Sensitivity,Maintenance,Scalability
0,"Nano - Material(0, 1)",0.000381,-0.081982,0.121212,-1.0,0.666667,0.333333,-0.5
1,"Nano - Material(0, 2)",-0.004762,0.409911,-0.242424,0.0,0.333333,0.333333,-0.5
2,"Nano - Material(0, 3)",-0.999619,-0.089087,-0.757576,0.0,-0.333333,1.000000,0.5
3,"Nano - Material(0, 4)",0.000000,-0.040991,0.242424,0.0,-0.333333,0.333333,0.0
4,"Nano - Material(0, 5)",0.000000,0.910913,-0.060606,-1.0,0.666667,0.333333,-0.5
5,"Nano - Material(0, 6)",0.000000,0.109310,0.000000,-1.0,0.333333,0.000000,0.0
6,"Nano - Material(1, 0)",-0.000381,0.081982,-0.121212,1.0,-0.666667,-0.333333,0.5
7,"Nano - Material(1, 2)",-0.005143,0.491893,-0.363636,1.0,-0.333333,0.000000,0.0
8,"Nano - Material(1, 3)",-1.000000,-0.007105,-0.878788,1.0,-1.000000,0.666667,1.0
9,"Nano - Material(1, 4)",-0.000381,0.040991,0.121212,1.0,-1.000000,0.000000,0.5


# Step 3 : Calculating Preference function

In [359]:
def relu(x):
    if x<0:
        return 0
    return x
def preference_function(df):
    df_n = df.copy()
    for col in df_n.columns:
        if col == 'Nano-materials':
            continue
        df_n[col] = df_n[col].apply(lambda x : relu(x))
    return df_n

In [360]:
pref_mat = preference_function(dif_mat)
pref_mat

,Nano-materials,Output (per sq. meter),Cost (per sq. meter),Durability,Efficiency,Sensitivity,Maintenance,Scalability
0,"Nano - Material(0, 1)",0.000381,0.000000,0.121212,0.0,0.666667,0.333333,0.0
1,"Nano - Material(0, 2)",0.000000,0.409911,0.000000,0.0,0.333333,0.333333,0.0
2,"Nano - Material(0, 3)",0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.5
3,"Nano - Material(0, 4)",0.000000,0.000000,0.242424,0.0,0.000000,0.333333,0.0
4,"Nano - Material(0, 5)",0.000000,0.910913,0.000000,0.0,0.666667,0.333333,0.0
5,"Nano - Material(0, 6)",0.000000,0.109310,0.000000,0.0,0.333333,0.000000,0.0
6,"Nano - Material(1, 0)",0.000000,0.081982,0.000000,1.0,0.000000,0.000000,0.5
7,"Nano - Material(1, 2)",0.000000,0.491893,0.000000,1.0,0.000000,0.000000,0.0
8,"Nano - Material(1, 3)",0.000000,0.000000,0.000000,1.0,0.000000,0.666667,1.0
9,"Nano - Material(1, 4)",0.000000,0.040991,0.121212,1.0,0.000000,0.000000,0.5


# Step 4 : Aggregate Preference Function

In [361]:
def aggregate_function(df,weights):
    df_n = df.copy()
    i=0
    for col in df.columns:
        if col == 'Nano-materials':
            df_n[col] = df_n[col]
            continue
        df_n[col] = df_n[col]*weights[i]
        i+=1
    return df_n

In [362]:
AHP_weights = [0.07,0.26, 0.19, 0.17, 0.13, 0.1, 0.08]
aggregate_mat = aggregate_function(pref_mat,AHP_weights)
aggregate_mat

,Nano-materials,Output (per sq. meter),Cost (per sq. meter),Durability,Efficiency,Sensitivity,Maintenance,Scalability
0,"Nano - Material(0, 1)",0.000027,0.000000,0.023030,0.00,0.086667,0.033333,0.00
1,"Nano - Material(0, 2)",0.000000,0.106577,0.000000,0.00,0.043333,0.033333,0.00
2,"Nano - Material(0, 3)",0.000000,0.000000,0.000000,0.00,0.000000,0.100000,0.04
3,"Nano - Material(0, 4)",0.000000,0.000000,0.046061,0.00,0.000000,0.033333,0.00
4,"Nano - Material(0, 5)",0.000000,0.236837,0.000000,0.00,0.086667,0.033333,0.00
5,"Nano - Material(0, 6)",0.000000,0.028420,0.000000,0.00,0.043333,0.000000,0.00
6,"Nano - Material(1, 0)",0.000000,0.021315,0.000000,0.17,0.000000,0.000000,0.04
7,"Nano - Material(1, 2)",0.000000,0.127892,0.000000,0.17,0.000000,0.000000,0.00
8,"Nano - Material(1, 3)",0.000000,0.000000,0.000000,0.17,0.000000,0.066667,0.08
9,"Nano - Material(1, 4)",0.000000,0.010658,0.023030,0.17,0.000000,0.000000,0.04


In [363]:
def sum_of_rows(df):
    l=[]
    df_n = df.copy()
    for i in range(df_n.shape[0]):
        l.append(sum(df_n.loc[i,]))
    df_n['Weighted'] = l
    return df_n

In [364]:
weighted_mat = aggregate_mat.copy()
weighted_mat = sum_of_rows(aggregate_mat.iloc[:,1:])

In [365]:
list_ = weighted_mat['Weighted']

# Step 4 : Forming Aggregated Matrix

In [366]:
def new_agg_mat_fun(df,list_,n):
    k=0
    new_agg_mat = df.copy()
    for i in range(n+1):
        for j in range(n+1):
            if i !=j:
                new_agg_mat.iloc[i,j] = list_[k]
                k+=1;
            else:
                new_agg_mat.iloc[i,j] = 0
    return new_agg_mat

In [367]:
new_agg_mat = new_agg_mat_fun(norm_matrix,list_,6)
new_agg_mat

,Output (per sq. meter),Cost (per sq. meter),Durability,Efficiency,Sensitivity,Maintenance,Scalability
0,0.000000,0.143057,0.183243,0.140000,0.079394,0.356837,0.071754
1,0.231315,0.000000,0.297892,0.316667,0.243688,0.258153,0.089736
2,0.086394,0.112784,0.000000,0.146667,0.132455,0.208473,0.086394
3,0.280409,0.368817,0.383925,0.000000,0.272478,0.592398,0.352163
4,0.053991,0.130027,0.203901,0.106667,0.000000,0.377495,0.125745
5,0.221515,0.034572,0.170000,0.316667,0.267576,0.000000,0.051515
6,0.170000,0.099724,0.281490,0.310000,0.249394,0.285084,0.000000


# Step 5 : calculating leaving and entering variables

In [368]:
def leaving_flow(df):
    df_n = df.copy()
    l=[]
    for i in range(df.shape[0]):
        l.append(round(sum(df_n.loc[i,])/len(df_n.loc[i,]),4))
    return l

In [369]:
def entering_flow(df):
    df_n = df.copy()
    l=[]
    for col in df_n.columns:
        l.append(round(sum(df[col])/df_n.shape[0],4))
    return l

In [370]:
def outrank():
    out_rank_df   = pd.DataFrame()
    out_rank_df['leaving_flow'] =  leaving_flow(new_agg_mat)
    out_rank_df['entering_flow'] = entering_flow(new_agg_mat)
    return out_rank_df

In [371]:
rank_mat = outrank()
rank_mat['Difference'] = rank_mat['leaving_flow']-rank_mat['entering_flow']

In [379]:
for col in df.columns:
    df.rename(columns = {col:col.strip()},inplace=True)

In [381]:
rank_mat['ranks'] = rank_mat['Difference'].rank(ascending=False).astype(int)
rank_mat['Material'] = df['Material']

In [384]:
# reordering
rank_mat = rank_mat[['Material','leaving_flow','entering_flow','Difference','ranks']]
rank_mat

,Material,leaving_flow,entering_flow,Difference,ranks
0,Piezoelectric,0.1392,0.1491,-0.0099,4
1,Triboelectric,0.2054,0.1270,0.0784,3
2,Thermoelectric,0.1105,0.2172,-0.1067,6
3,Photovoltaic,0.3215,0.1910,0.1305,1
4,Electroactive Polymers,0.1425,0.1779,-0.0354,5
5,Magnetoelectric,0.1517,0.2969,-0.1452,7
6,Pyroelectric,0.1994,0.1110,0.0884,2


# Promethee 1

In [385]:
def leaving_flow_pr1(df):
    df_n = df.copy()
    l=[]
    for i in range(df.shape[0]):
        l.append(round(sum(df_n.loc[i,]),4))
    return l

In [386]:
def entering_flow_pr1(df):
    df_n = df.copy()
    l=[]
    for col in df_n.columns:
        l.append(round(sum(df[col]),4))
    return l

In [387]:
def outrank_pr1():
    out_rank_df   = pd.DataFrame()
    out_rank_df['leaving_flow'] =  leaving_flow_pr1(new_agg_mat)
    out_rank_df['entering_flow'] = entering_flow_pr1(new_agg_mat)
    return out_rank_df

In [388]:
rank_mat_pr1 = outrank_pr1()

In [389]:
rank_mat_pr1

,leaving_flow,entering_flow
0,0.9743,1.0436
1,1.4375,0.8890
2,0.7732,1.5205
3,2.2502,1.3367
4,0.9978,1.2450
5,1.0618,2.0784
6,1.3957,0.7773
